# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Processamento de Linguagem Natural</font>

## Long Short-Term Memory Para Modelagem de Linguagem

### Geração Automática de Textos com Redes LSTM

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.6


In [2]:
!nvidia-smi

Sun May 24 00:40:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    On   | 00000000:05:00.0 Off |                  N/A |
| 23%   37C    P8     8W / 250W |    114MiB / 12194MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  On   | 00000000:09:00.0 Off |                  N/A |
| 23%   32C    P8     9W / 250W |      2MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [3]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [4]:
# Imports
import os
import csv
import math
import random
import collections
import zipfile
import numpy as np
import tensorflow as tf
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
%matplotlib inline

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

csv              1.0
numpy            1.18.4
matplotlib.pylab 1.18.4
tensorflow       2.2.0
Data Science Academy


In [6]:
# Obs: Este script está compatível com as versões 1.x e 2.x do TensorFlow.
# Optamos por manter assim, pois alguns recursos avançados usados neste script ainda não foram implementados no TF 2.

# Para executar este script com TF 2, nenhum passo adicional precisa ser feito.
# Para executar com TF 1, remova o prefixo tf.compat.v1 ao longo do scriipt e substitua por tf, e comente as 3 linhas abaixo.
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
tf.compat.v1.disable_eager_execution()

## Download do Dataset

Download de histórias, se não estiver presente no disco. Deve haver 100 arquivos ('stories/001.txt','stories/002.txt', ...)

In [7]:
url = 'https://www.cs.cmu.edu/~spok/grimmtmp/'

# Cria o diretório se necessário
dir_name = 'stories'
if not os.path.exists(dir_name):
    os.mkdir(dir_name)

def maybe_download(filename):
  print('Download do arquivo: ', dir_name+ os.sep+filename)

  if not os.path.exists(dir_name+os.sep+filename):
    filename, _ = urlretrieve(url + filename, dir_name+os.sep+filename)
  else:
    print('Arquivo ',filename, ' já existe.')

  return filename

num_files = 100
filenames = [format(i, '03d')+'.txt' for i in range(1,101)]

for fn in filenames:
    maybe_download(fn)

Download do arquivo:  stories/001.txt
Arquivo  001.txt  já existe.
Download do arquivo:  stories/002.txt
Arquivo  002.txt  já existe.
Download do arquivo:  stories/003.txt
Arquivo  003.txt  já existe.
Download do arquivo:  stories/004.txt
Arquivo  004.txt  já existe.
Download do arquivo:  stories/005.txt
Arquivo  005.txt  já existe.
Download do arquivo:  stories/006.txt
Arquivo  006.txt  já existe.
Download do arquivo:  stories/007.txt
Arquivo  007.txt  já existe.
Download do arquivo:  stories/008.txt
Arquivo  008.txt  já existe.
Download do arquivo:  stories/009.txt
Arquivo  009.txt  já existe.
Download do arquivo:  stories/010.txt
Arquivo  010.txt  já existe.
Download do arquivo:  stories/011.txt
Arquivo  011.txt  já existe.
Download do arquivo:  stories/012.txt
Arquivo  012.txt  já existe.
Download do arquivo:  stories/013.txt
Arquivo  013.txt  já existe.
Download do arquivo:  stories/014.txt
Arquivo  014.txt  já existe.
Download do arquivo:  stories/015.txt
Arquivo  015.txt  já exi

## Leitura dos Dados

Os dados serão armazenados em uma lista de listas em que cada lista representa um documento e o documento é uma lista de palavras. Vamos então quebrar o texto em bigramas.

In [8]:
def read_data(filename):

  with open(filename) as f:
    data = tf.compat.as_str(f.read())
    data = data.lower()
    data = list(data)
  return data

documents = []
for i in range(num_files):
    print('\nProcessando o arquivo %s'%os.path.join(dir_name,filenames[i]))
    chars = read_data(os.path.join(dir_name,filenames[i]))
    two_grams = [''.join(chars[ch_i:ch_i+2]) for ch_i in range(0,len(chars)-2,2)]
    documents.append(two_grams)
    print('Tamanho dos dados (Characters) (Documento %d) %d' %(i,len(two_grams)))
    print('String de exemplo (Documento %d) %s'%(i,two_grams[:50]))


Processando o arquivo stories/001.txt
Tamanho dos dados (Characters) (Documento 0) 3667
String de exemplo (Documento 0) ['in', ' o', 'ld', 'en', ' t', 'im', 'es', ' w', 'he', 'n ', 'wi', 'sh', 'in', 'g ', 'st', 'il', 'l ', 'he', 'lp', 'ed', ' o', 'ne', ', ', 'th', 'er', 'e ', 'li', 've', 'd ', 'a ', 'ki', 'ng', '\nw', 'ho', 'se', ' d', 'au', 'gh', 'te', 'rs', ' w', 'er', 'e ', 'al', 'l ', 'be', 'au', 'ti', 'fu', 'l,']

Processando o arquivo stories/002.txt
Tamanho dos dados (Characters) (Documento 1) 4928
String de exemplo (Documento 1) ['ha', 'rd', ' b', 'y ', 'a ', 'gr', 'ea', 't ', 'fo', 're', 'st', ' d', 'we', 'lt', ' a', ' w', 'oo', 'd-', 'cu', 'tt', 'er', ' w', 'it', 'h ', 'hi', 's ', 'wi', 'fe', ', ', 'wh', 'o ', 'ha', 'd ', 'an', '\no', 'nl', 'y ', 'ch', 'il', 'd,', ' a', ' l', 'it', 'tl', 'e ', 'gi', 'rl', ' t', 'hr', 'ee']

Processando o arquivo stories/003.txt
Tamanho dos dados (Characters) (Documento 2) 9745
String de exemplo (Documento 2) ['a ', 'ce', 'rt', 'ai', 'n ', 'f

Tamanho dos dados (Characters) (Documento 70) 3569
String de exemplo (Documento 70) ['th', 'er', 'e ', 'wa', 's ', 'on', 'ce', ' a', ' p', 'oo', 'r ', 'pe', 'as', 'an', 't ', 'wh', 'o ', 'ha', 'd ', 'no', ' l', 'an', 'd,', ' b', 'ut', ' o', 'nl', 'y ', 'a ', 'sm', 'al', 'l\n', 'ho', 'us', 'e,', ' a', 'nd', ' o', 'ne', ' d', 'au', 'gh', 'te', 'r.', '  ', 'th', 'en', ' s', 'ai', 'd ']

Processando o arquivo stories/072.txt
Tamanho dos dados (Characters) (Documento 71) 3793
String de exemplo (Documento 71) ['ab', 'ou', 't ', 'a ', 'th', 'ou', 'sa', 'nd', ' o', 'r ', 'mo', 're', ' y', 'ea', 'rs', ' a', 'go', ', ', 'th', 'er', 'e ', 'we', 're', ' i', 'n ', 'th', 'is', '\nc', 'ou', 'nt', 'ry', ' n', 'ot', 'hi', 'ng', ' b', 'ut', ' s', 'ma', 'll', ' k', 'in', 'gs', ', ', 'an', 'd ', 'on', 'e ', 'of', ' t']

Processando o arquivo stories/073.txt
Tamanho dos dados (Characters) (Documento 72) 5980
String de exemplo (Documento 72) ['th', 'er', 'e ', 'wa', 's ', 'on', 'ce', ' a', ' k', 'in', 'g ',

## Construindo os Dicionários (Bigrams)

Para entender cada um desses elementos, vamos também assumir o texto "Eu gosto de ir à escola"

* `dictionary`: mapeia uma palavra para um ID (i.e. {Eu:0, gosto:1, de:2, ir:3, à:4, escola:5})
* `reverse_dictionary`: mapeia um ID para uma palavra (i.e. {0:Eu, 1:gosto, 2:de, 3:ir, 4:à, 5:escola}
* `count`: Lista de elementos (palavra, frequência) (i.e. [(Eu,1),(gosto,1),(de,2),(ir,1),(à,1),(escola,1)]
* `data` : Contém a string de texto que lemos, onde palavras são substituídas por IDs de palavras (i.e. [0, 1, 2, 3, 2, 4])

Também introduzimos um token especial adicional `UNK` para indicar que palavras raras são muito raras para serem usadas.

In [9]:
def build_dataset(documents):
    chars = []
    # Esta vai ser uma lista de listas
    # onde a lista externa representa cada documento
    # e as listas internas representam palavras em um determinado documento
    data_list = []

    for d in documents:
        chars.extend(d)
    print('%d Caracteres encontrados.'%len(chars))
    count = []

    # Obter o bigrama classificado pela sua frequência (o mais alto vem primeiro)
    count.extend(collections.Counter(chars).most_common())

    # Cria um ID para cada bigrama dando o tamanho atual do dicionário
    # E adiciona esse item ao dicionário
    # Iniciar com 'UNK' atribuído a palavras muito raras
    dictionary = dict({'UNK':0})
    for char, c in count:

        # Apenas adicione um bigrama ao dicionário se sua frequência for maior que 10
        if c > 10:
            dictionary[char] = len(dictionary)

    unk_count = 0

    # Atravessar todo o texto que temos para substituir cada palavra da string pelo ID da palavra
    for d in documents:
        data = list()
        for char in d:

            # Se a palavra estiver no dicionário, use o ID da palavra, senão use o ID do token especial "UNK"
            if char in dictionary:
                index = dictionary[char]
            else:
                index = dictionary['UNK']
                unk_count += 1
            data.append(index)

        data_list.append(data)

    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data_list, count, dictionary, reverse_dictionary

global data_list, count, dictionary, reverse_dictionary,vocabulary_size

# Print de algumas estatísticas sobre os dados
data_list, count, dictionary, reverse_dictionary = build_dataset(documents)
print('Palavras mais comuns (+UNK)', count[:5])
print('Palavras menos comuns (+UNK)', count[-15:])
print('Dados de amostra', data_list[0][:10])
print('Dados de amostra', data_list[1][:10])
print('Vocabulário: ',len(dictionary))
vocabulary_size = len(dictionary)

# Para reduzir a memória.
del documents

449177 Caracteres encontrados.
Palavras mais comuns (+UNK) [('e ', 15229), ('he', 15164), (' t', 13443), ('th', 13076), ('d ', 10687)]
Palavras menos comuns (+UNK) [('bj', 1), ('ii', 1), ('i?', 1), ('z ', 1), ('c.', 1), ('"k', 1), ('pw', 1), ('f?', 1), (' z', 1), ('xq', 1), ('nm', 1), ('m?', 1), ('\t"', 1), ('\tw', 1), ('tz', 1)]
Dados de amostra [15, 28, 86, 23, 3, 95, 74, 11, 2, 16]
Dados de amostra [22, 156, 25, 37, 82, 185, 43, 9, 90, 19]
Vocabulário:  544


## Gerando Lotes de dados

O objeto a seguir gera um lote de dados que será usado para treinar a RNN. Mais especificamente, o gerador quebra uma dada sequência de palavras em segmentos `batch_size`. Também mantemos um cursor para cada segmento. Portanto, sempre que criamos um lote de dados, extraímos um item de cada segmento e atualizamos o cursor de cada segmento. 

Considere a seguinte frase: João e Maria foram ao cinema.

Com num_unroll = 5, temos os seguintes dados para gerar os lotes:

- Input 1 = 'J', 'o', 'ã', 'o', ''
- Output 1 = 'o', 'ã', 'o', '', 'e'


- Input 2 = 'e', '', 'M', 'a', 'r',
- Output 2 = '', 'M', 'a', 'r', 'i'

In [10]:
class DataGeneratorOHE(object):

    def __init__(self,text,batch_size,num_unroll):

        # Texto em que um bigrama é indicado por seu ID
        self._text = text

        # Número de bigramas no texto
        self._text_size = len(self._text)

        # Número de pontos de dados em um lote de dados
        self._batch_size = batch_size

        # num_unroll é o número de passos que nós desenrolamos a RNN em um único passo de treinamento
        # Esse é o número de etapas para as quais a entrada foi desenrolada, conforme discutido no método TBPTT.
        # Quanto maior esse número, maior a memória da RNN. No entanto, devido ao problema da dissipação do gradiente,
        # o efeito desse valor desaparece para valores num_unroll muito altos (acima de 50).
        # Observe que aumentar num_unroll também aumenta o requisito de memória do programa.
        self._num_unroll = num_unroll

        # Nós dividimos o texto em vários segmentos e o lote de dados é amostrado por
        # amostragem de um único item de um único segmento
        self._segments = self._text_size//self._batch_size
        self._cursor = [offset * self._segments for offset in range(self._batch_size)]

    def next_batch(self):

        # Inputs de treinamento (one-hot-encoded) e outputs de treinamento (one-hot-encoded)
        batch_data = np.zeros((self._batch_size,vocabulary_size),dtype=np.float32)
        batch_labels = np.zeros((self._batch_size,vocabulary_size),dtype=np.float32)

        # Preencha o ponto de dados em lote por ponto de dados
        for b in range(self._batch_size):

            # Se o cursor de um determinado segmento exceder o comprimento do segmento
            # redefinimos o cursor de volta ao início desse segmento
            if self._cursor[b]+1>=self._text_size:
                self._cursor[b] = b * self._segments

            # Adicionamos o texto no cursor como a entrada
            batch_data[b,self._text[self._cursor[b]]] = 1.0

            # Adicionamos o bigrama precedente como o rótulo a ser previsto
            batch_labels[b,self._text[self._cursor[b]+1]]= 1.0

            # Atualiza o cursor
            self._cursor[b] = (self._cursor[b]+1)%self._text_size

        return batch_data,batch_labels

    def unroll_batches(self):
        '''
        Isso produz uma lista de lotes num_unroll
        conforme exigido por uma única etapa de treinamento da RNN
        '''
        unroll_data,unroll_labels = [],[]
        for ui in range(self._num_unroll):
            data, labels = self.next_batch()
            unroll_data.append(data)
            unroll_labels.append(labels)

        return unroll_data, unroll_labels

    def reset_indices(self):
        '''
        Usado para redefinir todos os cursores, se necessário
        '''
        self._cursor = [offset * self._segments for offset in range(self._batch_size)]

# Executando um pequeno conjunto para ver se tudo está correto
dg = DataGeneratorOHE(data_list[0][25:50],5,5)
u_data, u_labels = dg.unroll_batches()

# Iterar através de cada lote de dados no conjunto desenrolado de lotes
for ui,(dat,lbl) in enumerate(zip(u_data,u_labels)):
    print('\n\nUnrolled index %d'%ui)
    dat_ind = np.argmax(dat,axis=1)
    lbl_ind = np.argmax(lbl,axis=1)
    print('\tInputs:')
    for sing_dat in dat_ind:
        print('\t%s (%d)'%(reverse_dictionary[sing_dat],sing_dat),end=", ")
    print('\n\tOutput:')
    for sing_lbl in lbl_ind:
        print('\t%s (%d)'%(reverse_dictionary[sing_lbl],sing_lbl),end=", ")



Unrolled index 0
	Inputs:
	e  (1), 	ki (131), 	 d (48), 	 w (11), 	be (70), 
	Output:
	li (98), 	ng (33), 	au (195), 	er (14), 	au (195), 

Unrolled index 1
	Inputs:
	li (98), 	ng (33), 	au (195), 	er (14), 	au (195), 
	Output:
	ve (41), 	
w (169), 	gh (106), 	e  (1), 	ti (112), 

Unrolled index 2
	Inputs:
	ve (41), 	
w (169), 	gh (106), 	e  (1), 	ti (112), 
	Output:
	d  (5), 	ho (62), 	te (61), 	al (84), 	fu (228), 

Unrolled index 3
	Inputs:
	d  (5), 	ho (62), 	te (61), 	al (84), 	fu (228), 
	Output:
	a  (82), 	se (58), 	rs (137), 	l  (57), 	l, (257), 

Unrolled index 4
	Inputs:
	a  (82), 	se (58), 	rs (137), 	l  (57), 	be (70), 
	Output:
	ki (131), 	 d (48), 	 w (11), 	be (70), 	au (195), 

## Definindo a LSTM

Este é um LSTM padrão. O LSTM possui 5 componentes principais.

* Cell state
* Hidden state
* Input gate
* Forget gate
* Output gate

Cada gate possui três conjuntos de pesos (1 conjunto para a entrada atual, 1 conjunto para o estado oculto anterior e 1 bias)

## Definindo Hiperparâmetros

Aqui definimos vários hiperparâmetros e são muito semelhantes aos que definimos no capítulo anterior. No entanto, adicionalmente, usamos dropout, uma técnica que ajuda a evitar overfitting.

In [11]:
# Número de neurônios nas variáveis de estado oculto
num_nodes = 128

# Número de pontos de dados em um lote que processamos
batch_size = 64

# Número de etapas de tempo para as quais nos desenrolamos durante a otimização
num_unrollings = 50

# Dropout
dropout = 0.0

# Salvando o arquivo
filename_extension = ''
if dropout > 0.0:
    filename_extension = '_dropout'

# Usado para salvar os valores de perplexidade
filename_to_save = 'lstm'+filename_extension+'.csv'

## Definindo Entradas e Saídas

No código, definimos dois tipos diferentes de entradas.

* Entradas de treinamento (as histórias que baixamos) (batch_size > 1 com unrolling)
* Entradas de validação (um conjunto de dados de validação) (bach_size = 1, sem unrolling)
* Entrada de teste (nova história que vamos gerar) (batch_size = 1, sem unrolling)

In [12]:
tf.compat.v1.reset_default_graph()

# Dados de entrada de treinamento
train_inputs, train_labels = [],[]

# Definindo entradas de treinamento "desenroladas" (unrolled)
for ui in range(num_unrollings):
    train_inputs.append(tf.compat.v1.placeholder(tf.float32, shape=[batch_size,vocabulary_size], name = 'train_inputs_%d'%ui))
    train_labels.append(tf.compat.v1.placeholder(tf.float32, shape=[batch_size,vocabulary_size], name = 'train_labels_%d'%ui))

# Espaços reservados de dados de validação
valid_inputs = tf.compat.v1.placeholder(tf.float32, shape=[1,vocabulary_size], name = 'valid_inputs')
valid_labels = tf.compat.v1.placeholder(tf.float32, shape=[1,vocabulary_size], name = 'valid_labels')

# Geração de texto: batch 1, sem desenrolar.
test_input = tf.compat.v1.placeholder(tf.float32, shape=[1, vocabulary_size], name = 'test_input')

## Definindo Parâmetros do Modelo

Agora nós definimos os parâmetros do modelo. Em comparação com as RNNs, as LSTMs possuem um grande número de parâmetros. Cada porta (entrada, esquecimento, memória e saída) possui três conjuntos diferentes de parâmetros.

In [13]:
# Input gate (i_t) - Quanta memória para gravar no estado da célula
# Conecta a entrada atual ao gate de entrada
ix = tf.Variable(tf.random.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))

# Conecta o estado oculto anterior ao gate de entrada
im = tf.Variable(tf.random.truncated_normal([num_nodes, num_nodes], stddev=0.02))

# Bias do portão de entrada
ib = tf.Variable(tf.random.uniform([1, num_nodes],-0.02, 0.02))

# Forget gate (f_t) - Quanta memória para descartar do estado da célula
# Conecta a entrada atual ao portão do esquecimento
fx = tf.Variable(tf.random.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))

# Conecta o estado oculto anterior ao portão do esquecimento
fm = tf.Variable(tf.random.truncated_normal([num_nodes, num_nodes], stddev=0.02))

# Bias da forget gate
fb = tf.Variable(tf.random.uniform([1, num_nodes],-0.02, 0.02))

# Valor candidato (c~ _t) - usado para calcular o estado atual da célula
# Conecta a entrada atual ao candidato
cx = tf.Variable(tf.random.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))

# Conecta o estado oculto anterior ao candidato
cm = tf.Variable(tf.random.truncated_normal([num_nodes, num_nodes], stddev=0.02))

# Bias do candidato
cb = tf.Variable(tf.random.uniform([1, num_nodes],-0.02,0.02))

# Porta de saída (o_t) - Quanta memória para saída do estado da célula
# Conecta a entrada atual ao gate de saída
ox = tf.Variable(tf.random.truncated_normal([vocabulary_size, num_nodes], stddev=0.02))

# Conecta o estado oculto anterior ao gate de saída
om = tf.Variable(tf.random.truncated_normal([num_nodes, num_nodes], stddev=0.02))

# Bias do gate de saída
ob = tf.Variable(tf.random.uniform([1, num_nodes],-0.02,0.02))

# Pesos e vieses do Softmax Classifier
w = tf.Variable(tf.random.truncated_normal([num_nodes, vocabulary_size], stddev=0.02))
b = tf.Variable(tf.random.uniform([vocabulary_size],-0.02,0.02))

# Variáveis que salvam o estado em desdobramentos.

# Hidden state
saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False, name='train_hidden')

# Cell state
saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False, name='train_cell')

# Mesmas variáveis para a fase de validação
saved_valid_output = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='valid_hidden')
saved_valid_state = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='valid_cell')

# Mesmas variáveis para fase de teste
saved_test_output = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='test_hidden')
saved_test_state = tf.Variable(tf.zeros([1, num_nodes]),trainable=False, name='test_cell')

## Definindo Cálculos da LSTM

Aqui primeiro definimos os cálculos da célula LSTM como uma função concisa. Em seguida, usamos essa função para definir a lógica de inferência de treinamento e tempo de teste.

In [14]:
# Definição do cálculo da célula.
def lstm_cell(i, o, state):
    """Cria a célula LSTM"""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

## Definindo a Lógica de Inferência

In [15]:
# =========================================================
# Lógica de inferência relacionada ao treinamento

# Mantém as saídas de estado calculadas em todos os desenrolamentos
# Usado para calcular a perda
outputs = list()

# Essas duas variáveis python são iterativamente atualizadas
# em cada etapa de desenrolamento
output = saved_output
state = saved_state

# Calcular o hidden state (saída) e o cell state (estado)
# recursivamente para todas as etapas do desenrolar
for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    output = tf.nn.dropout(output,rate=1 - (1.0-dropout))
    # Anexar cada valor de saída calculado
    outputs.append(output)

# Calcular os valores da pontuação (scores)
logits = tf.matmul(tf.concat(axis=0, values=outputs), w) + b

# Calcula previsões
train_prediction = tf.nn.softmax(logits)

# Computa a perplexidade do treinamento
train_perplexity_without_exp = tf.reduce_sum(input_tensor=tf.concat(train_labels,0)*-tf.math.log(tf.concat(train_prediction,0)+1e-10))/(num_unrollings*batch_size)


# ========================================================================
# Lógica de inferência relacionada à fase de validação

# Calcula a saída da célula LSTM para dados de validação
valid_output, valid_state = lstm_cell(valid_inputs, saved_valid_output, saved_valid_state)

# Calcula os logits
valid_logits = tf.compat.v1.nn.xw_plus_b(valid_output, w, b)

# Certifique-se de que as variáveis de estado estejam atualizadas
# antes de passar para a próxima iteração de geração
with tf.control_dependencies([saved_valid_output.assign(valid_output), saved_valid_state.assign(valid_state)]):
    valid_prediction = tf.nn.softmax(valid_logits)

# Computa perplexidade em validação
valid_perplexity_without_exp = tf.reduce_sum(input_tensor=valid_labels*-tf.math.log(valid_prediction+1e-10))


# ========================================================================
# Lógica de inferência relacionada à fase de teste

# Calcula a saída da célula LSTM para testar dados
test_output, test_state = lstm_cell(
test_input, saved_test_output, saved_test_state)

# Certifique-se de que as variáveis de estado estejam atualizadas
# antes de passar para a próxima iteração de geração
with tf.control_dependencies([saved_test_output.assign(test_output), saved_test_state.assign(test_state)]):
    test_prediction = tf.nn.softmax(tf.compat.v1.nn.xw_plus_b(test_output, w, b))

## Calculando a Perda (loss) da LSTM

Calculamos a perda de treinamento da LSTM aqui. É uma perda típica de entropia cruzada calculada sobre todas as pontuações obtidas para dados de treinamento ("loss").

In [16]:
# Antes de calcular a perda de treino,
# salvar o estado oculto e o estado da célula para
# suas respectivas variáveis do TensorFlow
with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):

    # Calcula a perda de treinamento concatenando os resultados de todas as etapas de tempo desenroladas
    loss = tf.reduce_mean(
      input_tensor=tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(axis=0, values=train_labels)))

## Definindo Taxa de Aprendizagem e o Otimizador com Clipping de Gradiente

Aqui nós definimos a taxa de aprendizado e o otimizador que vamos usar. Nós estaremos usando o otimizador Adam, pois é um dos melhores otimizadores por aí. Além disso, usamos recorte gradiente para evitar qualquer explosão gradiente.

In [17]:
# Learning rate decay
gstep = tf.Variable(0,trainable=False,name='global_step')

# Executar esta operação fará com que o valor de gstep
# aumente, reduzindo, por sua vez, a taxa de aprendizado
inc_gstep = tf.compat.v1.assign(gstep, gstep+1)

# Decaimento da Taxa de aprendizado toda vez que a gstep aumenta
tf_learning_rate = tf.compat.v1.train.exponential_decay(0.001,gstep,decay_steps=1, decay_rate=0.5)

# Adam Optimizer e gradient clipping
optimizer = tf.compat.v1.train.AdamOptimizer(tf_learning_rate)
gradients, v = zip(*optimizer.compute_gradients(loss))

# Clipping gradients
gradients, _ = tf.clip_by_global_norm(gradients, 5.0)

optimizer = optimizer.apply_gradients(zip(gradients, v))

## Redefinindo operações para redefinir estados ocultos

Às vezes, a variável de estado precisa ser redefinida (por exemplo, ao iniciar previsões no início de uma nova época)

In [18]:
# Reset train state
reset_train_state = tf.group(tf.compat.v1.assign(saved_state, tf.zeros([batch_size, num_nodes])),
                          tf.compat.v1.assign(saved_output, tf.zeros([batch_size, num_nodes])))

# Reset valid state
reset_valid_state = tf.group(tf.compat.v1.assign(saved_valid_state, tf.zeros([1, num_nodes])),
                          tf.compat.v1.assign(saved_valid_output, tf.zeros([1, num_nodes])))

# Reset test state
reset_test_state = tf.group(
    saved_test_output.assign(tf.random.normal([1, num_nodes],stddev=0.05)),
    saved_test_state.assign(tf.random.normal([1, num_nodes],stddev=0.05)))

## Amostragem Gananciosa (Greedy Sampling) para Quebrar a Repetição

Aqui escrevemos uma lógica simples para quebrar a repetição no texto. Especificamente, em vez de sempre obter a palavra que forneceu essa maior probabilidade de predição, amostramos aleatoriamente onde a probabilidade de ser selecionada é determinada por suas probabilidades de previsão.

In [19]:
def sample(distribution):
  '''
  Greedy Sampling
  Escolhemos as três melhores previsões dadas pela LSTM (com maiores probabilidades) e amostramos
  uma delas'''

  best_inds = np.argsort(distribution)[-3:]
  best_probs = distribution[best_inds]/np.sum(distribution[best_inds])
  best_idx = np.random.choice(best_inds,p=best_probs)
  return best_idx

## Executando a LSTM para gerar texto

Aqui nós treinamos a LSTM nos dados disponíveis e geramos texto usando a LSTM treinada para várias etapas. De cada documento extraímos texto para as etapas `steps_per_document` para treinar a LSTM. Também relatamos a perplexidade em treinamento ao final de cada etapa. Finalmente, testamos a LSTM pedindo que ela gere algum novo texto a partir de um bigrama escolhido aleatoriamente.

### Lógica do Learning Rate Decay

Aqui nós definimos a lógica para diminuir a taxa de aprendizado sempre que a perplexidade de validação não diminui

In [20]:
# Decaimento da taxa de aprendizado

# Se a perplexidade em validação não diminuir continuamente por várias épocas diminuímos a taxa de aprendizado
decay_threshold = 5

# Continue contando se a perplexidade aumenta
decay_count = 0

min_perplexity = 1e10

# Learning rate decay logic
def decay_learning_rate(session, v_perplexity):
  global decay_threshold, decay_count, min_perplexity

  # Decay learning rate
  if v_perplexity < min_perplexity:
    decay_count = 0
    min_perplexity= v_perplexity
  else:
    decay_count += 1

  if decay_count >= decay_threshold:
    print('\t Reduzindo a learning rate')
    decay_count = 0
    session.run(inc_gstep)

### Treinamento, Validação e Geração de Texto

Nós treinamos a LSTM em dados de treinamento existentes, verificamos a perplexidade de validação em um pedaço de texto e geramos um novo segmento de texto.

In [21]:
# Alguns hiperparâmetros necessários para o processo de treinamento

num_steps = 26
steps_per_document = 100
valid_summary = 1
train_doc_count = 100
docs_per_step = 10


# Captura o comportamento da perplexidade em treinamento ao longo do tempo
train_perplexity_ot = []
valid_perplexity_ot = []

session = tf.compat.v1.InteractiveSession()

# Initializing variables
tf.compat.v1.global_variables_initializer().run()
print('Variáveis Globais Inicializadas')

# Calcula a perda média em poucos passos
average_loss = 0

# Usamos os primeiros 10 documentos que possuem mais de 10 * bigramas steps_per_document
# para criar o conjunto de dados de validação

# Identifica os 10 primeiros documentos seguindo a condição acima
long_doc_ids = []
for di in range(num_files):
  if len(data_list[di])>10*steps_per_document:
    long_doc_ids.append(di)
  if len(long_doc_ids)==10:
    break

# Gerando dados de validação
data_gens = []
valid_gens = []
for fi in range(num_files):

  # Obter todos os bigramas se o ID do documento não estiver nos IDs do documento de validação
  if fi not in long_doc_ids:
    data_gens.append(DataGeneratorOHE(data_list[fi],batch_size,num_unrollings))

  # Se o documento estiver nos ids do documento de validação, somente atinja os últimos bigramas steps_per_document e
  # use os últimos bigramas steps_per_document como dados de validação
  else:
    data_gens.append(DataGeneratorOHE(data_list[fi][:-steps_per_document],batch_size,num_unrollings))

    # Definindo o gerador de dados de validação
    valid_gens.append(DataGeneratorOHE(data_list[fi][-steps_per_document:],1,1))


feed_dict = {}
for step in range(num_steps):

    print('Treinamento (Step: %d)'%step,end=' ')
    for di in np.random.permutation(train_doc_count)[:docs_per_step]:
        doc_perplexity = 0
        for doc_step_id in range(steps_per_document):

            # Obter um conjunto de lotes desenrolados
            u_data, u_labels = data_gens[di].unroll_batches()

            # Preencher o feed dict usando cada um dos lotes de dados presentes nos dados desenrolados
            for ui,(dat,lbl) in enumerate(zip(u_data,u_labels)):
                feed_dict[train_inputs[ui]] = dat
                feed_dict[train_labels[ui]] = lbl

            # Executando as operações do TensorFlow
            _, l, step_perplexity = session.run([optimizer, loss, train_perplexity_without_exp], feed_dict=feed_dict)

            # Atualizar a variável doc_perpelxity
            doc_perplexity += step_perplexity

            # Atualize a variável average_loss
            average_loss += step_perplexity

        # Mostra o progresso da impressão <train_doc_id_1>. <Train_doc_id_2>. ...
        print('(%d).'%di,end='')

    print('')

    # Gere novas amostras
    if (step+1) % valid_summary == 0:

      # Compute average loss
      average_loss = average_loss / (valid_summary*docs_per_step*steps_per_document)

      # Print losses
      print('Perda média no passo %d: %f' % (step+1, average_loss))
      print('\tPerplexidade no passo %d: %f' %(step+1, np.exp(average_loss)))
      train_perplexity_ot.append(np.exp(average_loss))

      # reset loss
      average_loss = 0
      valid_loss = 0

      # Calcular a perplexidade em validação
      for v_doc_id in range(10):

          # Lembre-se que processamos coisas como bigramas. Então precisamos dividir por 2
          for v_step in range(steps_per_document//2):
            uvalid_data,uvalid_labels = valid_gens[v_doc_id].unroll_batches()

            # Executar operações de TensorFlow relacionadas à fase de validação
            v_perp = session.run(
                valid_perplexity_without_exp,
                feed_dict = {valid_inputs:uvalid_data[0],valid_labels: uvalid_labels[0]}
            )

            valid_loss += v_perp

          session.run(reset_valid_state)

          # Redefinir o cursor do gerador de dados de validação
          valid_gens[v_doc_id].reset_indices()

      print()
      v_perplexity = np.exp(valid_loss/(steps_per_document*10.0//2))
      print("Perplexidade em Validação: %.2f\n"%v_perplexity)
      valid_perplexity_ot.append(v_perplexity)

      decay_learning_rate(session, v_perplexity)

      # Gerando novo texto ...
      # Nós estaremos gerando um segmento com 500 bigramas
      # Sinta-se à vontade para gerar vários segmentos mudando
      # o valor de segments_to_generate
      print('Texto gerado após a época %d ... '%step)
      segments_to_generate = 1
      chars_in_segment = 500

      for _ in range(segments_to_generate):
        print('======================== Novo Segmento de Texto ==========================')

        # Comece com uma palavra aleatória
        test_word = np.zeros((1,vocabulary_size),dtype=np.float32)
        rand_doc = data_list[np.random.randint(0,num_files)]
        test_word[0,rand_doc[np.random.randint(0,len(rand_doc))]] = 1.0
        print("\t",reverse_dictionary[np.argmax(test_word[0])],end='')

        # Gerando palavras dentro de um segmento, alimentando a previsão anterior
        # como a entrada atual de maneira recursiva
        for _ in range(chars_in_segment):
          sample_pred = session.run(test_prediction, feed_dict = {test_input:test_word})
          next_ind = sample(sample_pred.ravel())
          test_word = np.zeros((1,vocabulary_size),dtype=np.float32)
          test_word[0,next_ind] = 1.0
          print(reverse_dictionary[next_ind],end='')
        print("")

        # Reset train state
        session.run(reset_test_state)
        print('====================================================================')
      print("")

session.close()

# Gravar perplexidades de treinamento e validação em um arquivo csv
with open(filename_to_save, 'wt') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(train_perplexity_ot)
    writer.writerow(valid_perplexity_ot)

Variáveis Globais Inicializadas
Treinamento (Step: 0) (63).(77).(85).(66).(93).(27).(37).(18).(90).(48).
Perda média no passo 1: 4.326378
	Perplexidade no passo 1: 75.669706

Perplexidade em Validação: 67.52

Texto gerado após a época 0 ... 
======================== Novo Segmento de Texto ==========================
	 esut threat then and and him as ing the king, and had led the to dease, that then the king to her that at the to to their will, and then as ing the to come hat hall, and the to he king, and then the king then the king, and throuse, the kill, and the king and with the two the to then to through the celved to then the king then the two the to they to the king.  he the celse.  and he that the king then then and had to to the to that that they ing to the king to the king to had, and then then the king then the king they the king, and to they then to the king as the to ceased had had to ceese, the killed the crown, and had to dease, the to dease.  the king, and they to to the k

Treinamento (Step: 6) (99).(57).(12).(78).(80).(7).(13).(84).(3).(81).
Perda média no passo 7: 1.961193
	Perplexidade no passo 7: 7.107799

Perplexidade em Validação: 26.63

Texto gerado após a época 6 ... 
======================== Novo Segmento de Texto ==========================
	 ey came tatingthe
like the shoemaker to the tailor, and well the came to be to the tailor and wanted to it.  "then tailor, but it stailor, and stood will not was a tailor was to serving, and worn was still into his fatest and has beent to gat in into the took the for the
walling to him.  the said he his cand will go that the was.  the took the tandor, and whet him into the royal tailor and a shoemaker.  "i have go down in the straach stair to the courter.  then he began to was a tale for my could not merrow, the well and sto the stork with
her inst of the tailor with
her inst they the made and at one of
the shoemaker he had began to the candle had into he had that in, and wholevery time
did not find it, and

Treinamento (Step: 12) (89).(67).(6).(79).(24).(46).(62).(16).(18).(69).
Perda média no passo 13: 1.624810
	Perplexidade no passo 13: 5.077453

Perplexidade em Validação: 34.10

Texto gerado após a época 12 ... 
======================== Novo Segmento de Texto ==========================
	 hould
married that, and then they sat down were the dres, but what he had bested its
have not begate to set hich for here call out of the glass.

when he had be carried the ravends again, and sat down who he had found the thor, and said,
the man took her but to distantle.

when she had gone
out, and the stick sate to be all bed instance.  said th, you have not have
you
do no set about the maidh ons brothers, which was head to the glass brothers, and when they went of my lose it, and brink, but her did not get fas
the heads, put it which the had and was
speak, had
 and they do her day by her belion.  then the maiden to take and every they that stake of you have never the draven.

he said,
it was
hich
th

Treinamento (Step: 18) (97).(49).(22).(86).(30).(46).(70).(27).(69).(66).
Perda média no passo 19: 2.064837
	Perplexidade no passo 19: 7.884014

Perplexidade em Validação: 23.89

Texto gerado após a época 18 ... 
======================== Novo Segmento de Texto ==========================
	 o
god for him, and when a golden, he was away, but it our nothing, and will below into the door, that was haated that on the white was bring to have to them, and at as once he had to coman, and said, god husband.

then he had
not diving as the children, but the thandfather her bear in asted to them, and when the princessed and said her to third to see the kingdom to be
his little, were must off in the world.  then the wolf.  he saw that he had found no one day throw how forth, he cried on the golden chald with them.

then his heart as she came out, and said went with them, and the godfather, took he was to below, and is to be to each other, and then he said to the door, and said
he had gone in the wor

Treinamento (Step: 24) (41).(57).(71).(45).(85).(82).(95).(12).(87).(62).
Perda média no passo 25: 1.751499
	Perplexidade no passo 25: 5.763236

Perplexidade em Validação: 34.46

Texto gerado após a época 24 ... 
======================== Novo Segmento de Texto ==========================
	 eed that in the waters and pieced to do.

then he took it is any of the court, and it and were this, but he was mercinagain, and over what he had come aaced han brought by ame, and at all die better on the wedding to
seated him to lost that she was of snow, then the sun came out, and he was coarrous.  of the man would just light, and said, then had had to sento my droveress was foress, and is and round, but if you are you had golden and parget in took at the gretched, and however, i trod once at the har was if were, but before so empay, but she down the old contre an the merrowed it was merry, and when i brought the midden brousers, who kasped, and the mouth and said, yes, i am not to the glass, of it

## LSTM com Beam-Search (segundo modelo LSTM)

Aqui, alteramos as operações do TensorFlow relacionadas à predição, previamente definidas, para empregar a busca beam-search. A beam-search é uma maneira de prever vários passos de tempo à frente. Em vez de usar a melhor previsão que temos em um dado intervalo de tempo, obtemos predições para várias etapas de tempo e obtemos a sequência de maior probabilidade conjunta.

In [22]:
# Número de etapas para "olhar" a frente
beam_length = 5

# Número de vizinhos para comparar em cada etapa
beam_neighbors = 5

# Nós redefinimos a geração de amostra com beam search
sample_beam_inputs = [tf.compat.v1.placeholder(tf.float32, shape=[1, vocabulary_size]) for _ in range(beam_neighbors)]

best_beam_index = tf.compat.v1.placeholder(shape=None, dtype=tf.int32)
best_neighbor_beam_indices = tf.compat.v1.placeholder(shape=[beam_neighbors], dtype=tf.int32)

# Mantém a saída de cada feixe (beam)
saved_sample_beam_output = [tf.Variable(tf.zeros([1, num_nodes])) for _ in range(beam_neighbors)]

# Mantém o estado de cada feixe (beam)
saved_sample_beam_state = [tf.Variable(tf.zeros([1, num_nodes])) for _ in range(beam_neighbors)]

# Redefinindo os estados do feixe de amostra (deve ser feito no início de cada geração de fragmentos de texto)
reset_sample_beam_state = tf.group(
    *[saved_sample_beam_output[vi].assign(tf.zeros([1, num_nodes])) for vi in range(beam_neighbors)],
    *[saved_sample_beam_state[vi].assign(tf.zeros([1, num_nodes])) for vi in range(beam_neighbors)]
)

# Nós empilhamos para realizar a operação de coleta abaixo
stacked_beam_outputs = tf.stack(saved_sample_beam_output)
stacked_beam_states = tf.stack(saved_sample_beam_state)

# Os estados do feixe para cada feixe precisam ser atualizados a cada profundidade da árvore
# Considere um exemplo em que você tem 3 classes onde obtemos os dois melhores vizinhos (marcados com estrela)
#     a`      b*       c
#   / | \   / | \    / | \
#  a  b c  a* b` c  a  b  c
# Já que ambos os candidatos do nível 2 vêm do pai b
# Precisamos atualizar os dois estados / saídas de saved_sample_beam_state / output para ter o índice 1 (correspondente ao pai b)
update_sample_beam_state = tf.group(
    *[saved_sample_beam_output[vi].assign(tf.gather_nd(stacked_beam_outputs,[best_neighbor_beam_indices[vi]])) for vi in range(beam_neighbors)],
    *[saved_sample_beam_state[vi].assign(tf.gather_nd(stacked_beam_states,[best_neighbor_beam_indices[vi]])) for vi in range(beam_neighbors)]
)

# Calculamos o estado lstm_cell e a saída para cada feixe
sample_beam_outputs, sample_beam_states = [],[]
for vi in range(beam_neighbors):
    tmp_output, tmp_state = lstm_cell(
        sample_beam_inputs[vi], saved_sample_beam_output[vi], saved_sample_beam_state[vi]
    )
    sample_beam_outputs.append(tmp_output)
    sample_beam_states.append(tmp_state)

# Para um determinado conjunto de beams, gera uma lista de vetores de previsão de tamanho beam_neighbors
# cada feixe com as previsões de vocabulário completo
sample_beam_predictions = []
for vi in range(beam_neighbors):
    with tf.control_dependencies([saved_sample_beam_output[vi].assign(sample_beam_outputs[vi]),
                                saved_sample_beam_state[vi].assign(sample_beam_states[vi])]):
        sample_beam_predictions.append(tf.nn.softmax(tf.compat.v1.nn.xw_plus_b(sample_beam_outputs[vi], w, b)))

## Executando a LSTM com Beam Search para Gerar Texto

Aqui nós treinamos a LSTM nos dados disponíveis e geramos texto usando a LSTM treinada para várias etapas. De cada documento extraímos texto para as etapas `steps_per_document` para treinar a LSTM. Também relatamos a perplexidade em treinamento ao final de cada etapa. Finalmente, testamos a LSTM, solicitando que ela gere algum novo texto com beam search a partir de um bigrama escolhido aleatoriamente.

### Lógica do Learning rate Decay 

Aqui nós definimos a lógica para diminuir a taxa de aprendizado sempre que a perplexidade de validação não diminui

In [23]:
# Decaimento da taxa de aprendizado

# Se a perplexidade em validação não diminuir continuamente por várias épocas diminuímos a taxa de aprendizado
decay_threshold = 5

# Continue contando quantas vezes a perplexidade aumenta
decay_count = 0

min_perplexity = 1e10

# Learning rate decay logic
def decay_learning_rate(session, v_perplexity):
  global decay_threshold, decay_count, min_perplexity

  # Decay learning rate
  if v_perplexity < min_perplexity:
    decay_count = 0
    min_perplexity= v_perplexity
  else:
    decay_count += 1

  if decay_count >= decay_threshold:
    print('\t Reduzindo a learning rate')
    decay_count = 0
    session.run(inc_gstep)

### Definindo a lógica de previsão de feixe (beam)

Aqui nós definimos a função que toma a sessão como um argumento e produz um feixe de previsões

In [24]:
test_word = None

def get_beam_prediction(session):

    # Gerando palavras dentro de um segmento com Beam Search
    # Para tornar alguns cálculos mais claros, usamos o exemplo da seguinte forma
    # Temos três classes com beam_neighbors = 2 (melhor candidato indicado por *)
    # Para simplificar, assumimos que o melhor candidato sempre tem probabilidade de 0,5 na previsão de saída
    # segundo melhor tem previsão de saída de 0,2
    #           a`                   b*                   c                <--- root level
    #    /     |     \         /     |     \        /     |     \
    #   a      b      c       a*     b`     c      a      b      c         <--- depth 1
    # / | \  / | \  / | \   / | \  / | \  / | \  / | \  / | \  / | \
    # a b c  a b c  a b c   a*b c  a`b c  a b c  a b c  a b c  a b c       <--- depth 2
    # Então as melhores beams na profundidade 2 seriam
    # b-a-a e b-b-a

    global test_word
    global sample_beam_predictions
    global update_sample_beam_state

    # Calcula os candidatos no nível raiz
    feed_dict = {}
    for b_n_i in range(beam_neighbors):
        feed_dict.update({sample_beam_inputs[b_n_i]: test_word})

    # Calculamos as previsões de amostra para todos os vizinhos com a mesma palavra / caractere inicial
    # Isso é importante para atualizar o estado de todas as instâncias da pesquisa de feixes
    sample_preds_root = session.run(sample_beam_predictions, feed_dict = feed_dict)
    sample_preds_root = sample_preds_root[0]

    # Indices of top-k candidates
    # b e a em nosso exemplo (root level)
    this_level_candidates =  (np.argsort(sample_preds_root,axis=1).ravel()[::-1])[:beam_neighbors].tolist()

    # Probabilidades of top-k candidates
    # 0.5 e 0.2
    this_level_probs = sample_preds_root[0,this_level_candidates]

    # Atualizar a sequência de teste produzida por cada feixe a partir do cálculo do nível da raiz
    # Sequência de teste parece para o nosso exemplo (na raiz)
    # [BA]
    test_sequences = ['' for _ in range(beam_neighbors)]
    for b_n_i in range(beam_neighbors):
        test_sequences[b_n_i] += reverse_dictionary[this_level_candidates[b_n_i]]

    # Faça os cálculos para o resto da profundidade da árvore de busca de beams
    for b_i in range(beam_length-1):

        # Palavras candidatas para cada beam
        test_words = []

        # Palavras previstas de cada beam
        pred_words = []

        # Computando feed_dict para a pesquisa de feixes (exceto raiz)
        # feed dict deve conter as melhores palavras / chars / bigramas encontradas no nível anterior de pesquisa

        # Para o nível 1 em nosso exemplo, isso seria
        # sample_beam_inputs[0]: b, sample_beam_inputs[1]:a
        feed_dict = {}
        for p_idx, pred_i in enumerate(this_level_candidates):

            # Atualizando o feed_dict para obter as próximas previsões
            test_words.append(np.zeros((1,vocabulary_size),dtype=np.float32))
            test_words[p_idx][0,this_level_candidates[p_idx]] = 1.0

            feed_dict.update({sample_beam_inputs[p_idx]:test_words[p_idx]})

        # Calculando previsões para todos os vizinhos em beams
        # Esta é uma lista de vetores onde cada vetor é o vetor de previsão para um certo feixe
        # Para o nível 1 em nosso exemplo, os valores de previsão para
        #      b             a  (resultados de pesquisa de feixes anteriores)
        # [a,  b,  c],  [a,  b,  c] (previsões de nível atual) seria
        # [0.1,0.1,0.1],[0.5,0.2,0]
        sample_preds_all_neighbors = session.run(sample_beam_predictions, feed_dict=feed_dict)

        # Crie um único vetor com
        # Fazendo o nosso exemplo [0.1,0.1,0.1,0.5,0.2,0]
        sample_preds_all_neighbors_concat = np.concatenate(sample_preds_all_neighbors,axis=1)

        # Atualizar this_level_candidates para ser usado na próxima iteração
        # E atualize as probabilidades para cada feixe
        # No nosso exemplo, estes seriam [3,4] (índices com valor máximo do vetor acima)
        this_level_candidates = np.argsort(sample_preds_all_neighbors_concat.ravel())[::-1][:beam_neighbors]

        # No exemplo, isso seria [1,1]
        parent_beam_indices = this_level_candidates//vocabulary_size

        # Normaliza this_level_candidates para ficar entre [0, vocabulary_size]
        # Neste exemplo, isso seria [0,1]
        this_level_candidates = (this_level_candidates%vocabulary_size).tolist()

        # Aqui nós atualizamos o estado final de cada feixe a ser
        # o estado que estava no índice 1. Porque, para os dois candidatos nesse nível, o pai é
        # no índice 1 (que é b do nível raiz)
        session.run(update_sample_beam_state, feed_dict={best_neighbor_beam_indices: parent_beam_indices})

        # Aqui nós atualizamos as probabilidades conjuntas de cada beam e adicionamos os candidatos recém-encontrados à sequência

        # Este é atualmente [0,5,0,2]
        tmp_this_level_probs = np.asarray(this_level_probs)

        # Isto é atualmente [b, a]
        tmp_test_sequences = list(test_sequences)

        for b_n_i in range(beam_neighbors):
            # Nós fazemos com que o elemento b_n_i de this_level_probs seja a probabilidade dos pais
            # No exemplo, os índices pai são [1,1]
            # Então this_level_probs se torna [0.5,0.5]
            this_level_probs[b_n_i] = tmp_this_level_probs[parent_beam_indices[b_n_i]]

            # Em seguida, multiplicamos este nós pelas probabilidades dos melhores candidatos a partir do nível atual
            # [0,5 * 0,5, 0,5 * 0,2] = [0,25,0,1]
            this_level_probs[b_n_i] *= sample_preds_all_neighbors[parent_beam_indices[b_n_i]][0,this_level_candidates[b_n_i]]

            # Torne o elemento b_n_i de test_sequences como o pai correto dos melhores candidatos atuais
            # No exemplo, isso se torna [b, b]
            test_sequences[b_n_i] = tmp_test_sequences[parent_beam_indices[b_n_i]]

            # Agora, anexamos os melhores candidatos atuais
            # Neste exemplo, isso se torna [ba, bb]
            test_sequences[b_n_i] += reverse_dictionary[this_level_candidates[b_n_i]]

            # Criar uma representação codificada a quente para cada candidato
            pred_words.append(np.zeros((1,vocabulary_size),dtype=np.float32))
            pred_words[b_n_i][0,this_level_candidates[b_n_i]] = 1.0

    # Calcule a melhor identificação de feixe com base na maior probabilidade de feixe
    # Usando a maior probabilidade de feixe sempre leva a texto muito monótono
    # Deixe-nos provar um aleatoriamente onde um sendo amostrado é decidido pela probabilidade de que feixe
    rand_cand_ids = np.argsort(this_level_probs)[-3:]
    rand_cand_probs = this_level_probs[rand_cand_ids]/np.sum(this_level_probs[rand_cand_ids])
    random_id = np.random.choice(rand_cand_ids,p=rand_cand_probs)

    best_beam_id = parent_beam_indices[random_id]

    # Atualizar variáveis de estado e saída para previsão de teste
    session.run(update_sample_beam_state,feed_dict={best_neighbor_beam_indices:[best_beam_id for _ in range(beam_neighbors)]})

    # Faça a última palavra / caracter / bigram do melhor feixe
    test_word = pred_words[best_beam_id]

    return test_sequences[best_beam_id]

### Treinamento, Validação e Geração de Texto

Nós treinamos a LSTM em dados de treinamento existentes, verificamos a perplexidade de validação em um pedaço de texto  e geramos um novo segmento de texto

In [25]:
# Arquivos para salvar o modelo
filename_to_save = 'lstm_beam_search_dropout'

# Alguns hiperparâmetros necessários para o processo de treinamento
num_steps = 26
steps_per_document = 100
valid_summary = 1
train_doc_count = 100
docs_per_step = 10


beam_nodes = []

beam_train_perplexity_ot = []
beam_valid_perplexity_ot = []
session = tf.compat.v1.InteractiveSession()

tf.compat.v1.global_variables_initializer().run()

print('Inicializado')
average_loss = 0

# Usamos os primeiros 10 documentos que possuem mais de 10 * bigramas steps_per_document
# para criar o conjunto de dados de validação

# Identifica os 10 primeiros documentos seguindo a condição acima
long_doc_ids = []
for di in range(num_files):
  if len(data_list[di])>10*steps_per_document:
    long_doc_ids.append(di)
  if len(long_doc_ids)==10:
    break

# Gerando dados de validação
data_gens = []
valid_gens = []
for fi in range(num_files):

  # Obter todos os bigramas se o ID do documento não estiver nos IDs do documento de validação
  if fi not in long_doc_ids:
    data_gens.append(DataGeneratorOHE(data_list[fi],batch_size,num_unrollings))

  # Se o documento estiver nos ids do documento de validação, somente atinja os últimos bigramas
  # steps_per_document e use os últimos bigramas steps_per_document como dados de validação
  else:
    data_gens.append(DataGeneratorOHE(data_list[fi][:-steps_per_document],batch_size,num_unrollings))

    # Definindo o gerador de dados de validação
    valid_gens.append(DataGeneratorOHE(data_list[fi][-steps_per_document:],1,1))


feed_dict = {}
for step in range(num_steps):

    for di in np.random.permutation(train_doc_count)[:docs_per_step]:
        doc_perplexity = 0
        for doc_step_id in range(steps_per_document):

            # Obter um conjunto de lotes desenrolados
            u_data, u_labels = data_gens[di].unroll_batches()

            # Preencher o feed dict usando cada um dos lotes de dados presentes nos dados desenrolados
            for ui,(dat,lbl) in enumerate(zip(u_data,u_labels)):
                feed_dict[train_inputs[ui]] = dat
                feed_dict[train_labels[ui]] = lbl

            # Executando as operações do TensorFlow
            _, l, step_perplexity = session.run([optimizer, loss, train_perplexity_without_exp], feed_dict=feed_dict)

            # Atualiza a variável doc_perpelxity
            doc_perplexity += step_perplexity

            # Atualiza a variável average_loss
            average_loss += step_perplexity

        # Mostra o progresso da impressão <train_doc_id_1>. <Train_doc_id_2>. ...
        print('(%d).'%di,end='')

    print('')

    if (step+1) % valid_summary == 0:

      # Compute average loss
      average_loss = average_loss / (docs_per_step*steps_per_document*valid_summary)

      # Print loss
      print('Average loss at step %d: %f' % (step+1, average_loss))
      print('\tPerplexity at step %d: %f' %(step+1, np.exp(average_loss)))
      beam_train_perplexity_ot.append(np.exp(average_loss))

      average_loss = 0

      valid_loss = 0

      # Calcula a perplexidade em validação
      for v_doc_id in range(10):

          # Lembre-se que processamos coisas como bigramas. Então precisamos dividir por 2
          for v_step in range(steps_per_document//2):
            uvalid_data,uvalid_labels = valid_gens[v_doc_id].unroll_batches()

            # Executar operações do TensorFlow relacionadas à fase de validação
            v_perp = session.run(
                valid_perplexity_without_exp,
                feed_dict = {valid_inputs:uvalid_data[0],valid_labels: uvalid_labels[0]}
            )

            valid_loss += v_perp

          session.run(reset_valid_state)

          # Redefinir o cursor do gerador de dados de validação
          valid_gens[v_doc_id].reset_indices()

      print()
      v_perplexity = np.exp(valid_loss/(steps_per_document*10.0//2))
      print("Perplexidade em Validação: %.2f\n"%v_perplexity)
      beam_valid_perplexity_ot.append(v_perplexity)

      # Decay learning rate
      decay_learning_rate(session, v_perplexity)

      # Gerando novo texto ...
      # Nós estaremos gerando um segmento com 500 bigramas
      # Sinta-se à vontade para gerar vários segmentos mudando
      # o valor de segments_to_generate
      print('Texto gerado após a época %d ... '%step)
      segments_to_generate = 1
      chars_in_segment = 500//beam_length

      for _ in range(segments_to_generate):
        print('======================== Novo Segmento de Texto ==========================')

        # Primeira palavra gerada aleatoriamente
        test_word = np.zeros((1,vocabulary_size),dtype=np.float32)
        rand_doc = data_list[np.random.randint(0,num_files)]
        test_word[0,rand_doc[np.random.randint(0,len(rand_doc))]] = 1.0
        print("",reverse_dictionary[np.argmax(test_word[0])],end='')

        for _ in range(chars_in_segment):

            test_sequence = get_beam_prediction(session)
            print(test_sequence,end='')

        print("")
        session.run([reset_sample_beam_state])

        print('====================================================================')
      print("")

session.close()

with open(filename_to_save, 'wt') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(beam_train_perplexity_ot)
    writer.writerow(beam_valid_perplexity_ot)

Inicializado
(79).(22).(21).(15).(76).(11).(12).(72).(88).(54).
Average loss at step 1: 4.404003
	Perplexity at step 1: 81.777572

Perplexidade em Validação: 69.13

Texto gerado após a época 0 ... 
======================== Novo Segmento de Texto ==========================
 ngy to to they to the king, and the king, the king at the king the cat to the king said the king, and the king, the king at the king sainly the king and said the king, and the king, the king at the king and the would to the king, answer to the king was cat the soot to she could to the king with he was said the king, and the king, the king at the king and the would the clace, when, and the king, and the king, the king at the king the cat the could wat the wat the was shat the king and the cat the could to the king, answer to the king was cat the soot to to the king, the king at the king the cat the could wat the wat the was shat the king and the cat the could to ther was she could to the king, the king and said the ki

the king had some a clock an and said, into the moon to thorent, and made thould he was ther all town here if she bed one after so them that one one come to home, but through the took, and when the self be himselves that she wast into the wall, and crince strance, and in the stones in his
stomach will down somethan trouble of eaters, ther she would not got teach othought, and had over hers to the wedding.
and
when he got to east the tood thought of the world got teat and thought came in a great trought head, the plach were stoet it was still atered.  took the women him them, and save, they somet poor children, began to have and themself it on the breading, he had nother.  threw int in the moutiful one one lith as one out one one childen all the the wolf wish and stoopeds and as miller and belong so as allowfully he wand told home and feop over him as a stokes the there.  then he behild in the towed and steplive behtimed the fathing-mally howe. 

(11).(77).(6).(41).(81).(91).(61).(35).(

youth.  the youth called haved, and then said them, that they saw them, and when take you.   the secred, so that they everything had some them, and he came to gother.  i shall said he, the man walked to theirward, and said, "he replied, that they will sonot belive the man still said to helses, he have stand, you shaly, and had no not reach of strength.  in he came that he willed in the old woman said that they were likewise what he hammediantly road, and
then the cannots of wife, and that the young whilen themsell
into the right them to so that, and asket was
devired their, now out of yourn as she live." then they had walked from all them, begain whis, and they went through the wolf all to

(68).(49).(65).(26).(87).(12).(69).(99).(58).(23).
Average loss at step 14: 1.822478
	Perplexity at step 14: 6.187170

Perplexidade em Validação: 41.61

Texto gerado após a época 13 ... 
======================== Novo Segmento de Texto ==========================
 omently
hans.  what did not fire ming

her, and carried a hunger.  when they came, and put on it king and said to him and where when he would like you shall looking, and when then they well.  when it anone would have the young maiden was away, and she said, you are took it was seized her.

the kiddle, thousaid to her mother, and that took it the fore to as heard that the belongs was a ring which, and she had no
one
thought to the king's daughter.  then it ask, he said to him and the three him out of his.  then two fox's plated his knapsack, where thereupon the king's dauted her haat, and thought the king's daughter, and asked his through to that at 

(49).(72).(51).(57).(26).(74).(75).(32).(64).(91).
Average loss at step 20: 2.115284
	Perplexity at step 20: 8.291943

Perplexidade em Validação: 25.37

Texto gerado após a época 19 ... 
======================== Novo Segmento de Texto ==========================
 d, and there there is not being time, then he saw their death, said he, for you, after the lorded and bride, their do

heird, his shape, and when he has brought he had to herself on his hand to his head, and then went with
her.  now, not whom he had gold with home in the morning, the other the maiden to self on, but and she went for them.  and take her father should not stretcher little than the wolf, what is in hall looking his father, what is you, and reserlen." then they carriage.  when he went out to had great death, and put away a blood for spily again.  then only whom he came to her eyes,

(70).(42).(34).(79).(11).(85).(86).(20).(4).(58).
Average loss at step 26: 1.889189
	Perplexity at step 26: 6.614000

Perplexidade em Validação: 23.54

Texto gerado após a época 25 ... 
======================== Novo Segmento de Texto ==========================
 d before the town the givensurned him the bride, and when the princess, who had to put in heard.  when your man in the fathidding there was no longer anything, if anything said to them beautif take, and they went into the children when the king was not i

# Fim